In [1]:
import pandas as pd
import zipfile
import os
import sys
from datetime import datetime
from datetime import timedelta
import numpy as np
pd.set_option('display.max_columns', 500)

In [2]:
# dateparse = lambda x: pd.datetime.strptime(x, '%Y-%m-%d')
vars_serg = ["estadoOrden","Ciudad","Departamento",
             "stado_producto","Marca","Pasillo","Categoria",
             "ProductoEnPromocion","cantidadVendida",
             "TotalPagado","Domicilio","PrecioFinal",
             "FechaCreaciónOrden"]

In [3]:
import dask.dataframe as dd

In [5]:
df_dask = dd.read_csv("../Data/MERQ_CONS",
                      usecols=vars_serg,
                      encoding='latin1',
                      assume_missing=True,
                     dtype={'estadoOrden' : str,
                            'TotalPagado' : np.float64,
                            "Domicilio" : str,
                            "FechaCreaciónOrden" : str,
                            "FechaProgramaciónOrden" : str,
                            "FechadeEntregaOrden" : str,
                            "Ciudad" : str,
                            "fechacreacionProducto" : str,
                            "stado_producto" : str,
                            "Marca" : str,
                            "Departamento" : str,
                            "Pasillo" : str,
                            "Categoria" : str,
                            "ProductoEnPromocion" : np.float64,
                            "cantidadVendida" : np.float64,
                            "PrecioFinal": np.float64})

In [6]:
df_dask = df_dask.dropna()
df_dask.set_index("FechaCreaciónOrden")
df_dask["FechaCreaciónOrden"] = df_dask["FechaCreaciónOrden"].map_partitions(pd.to_datetime,
                                                                             format='%Y-%m-%d',
                                                                             meta = ('datetime64[ns]'))encoding='latin1'

In [7]:
df_dask.head()

,estadoOrden,TotalPagado,Domicilio,FechaCreaciónOrden,Ciudad,stado_producto,Marca,Departamento,Pasillo,Categoria,cantidadVendida,ProductoEnPromocion,PrecioFinal
0,Cancelled,10450.0,4000,2017-02-13,Bogotá,Pending,Justo&Bueno,Bebidas,Té,Té preparado,2.0,0.0,950.0
1,Cancelled,10450.0,4000,2017-02-13,Bogotá,Pending,Justo&Bueno,Bebidas,Té,Té preparado,5.0,0.0,950.0
2,Cancelled,10450.0,4000,2017-02-13,Bogotá,Pending,Justo&Bueno,Bebidas,Té,Té preparado,4.0,0.0,950.0
3,Cancelled,13650.0,2000,2017-02-13,Bogotá,Pending,Merqueo,Lácteos y huevos,Huevos,Huevos,1.0,1.0,6900.0
4,Cancelled,13650.0,2000,2017-02-13,Bogotá,Pending,Merqueo,Lácteos y huevos,Huevos,Huevos,1.0,0.0,3750.0


In [8]:
vars_group_b = ["FechaCreaciónOrden",
                "Ciudad","Departamento",
                "estadoOrden","stado_producto",
                "Marca","Pasillo","Categoria",
                "ProductoEnPromocion"]

In [9]:
%time agg = df_dask.groupby(vars_group_b).sum()

CPU times: user 39.2 ms, sys: 0 ns, total: 39.2 ms
Wall time: 38.8 ms


In [10]:
df_p = pd.read_csv("../Data/MERQ_CONS",
                      usecols=vars_serg,
                      encoding='latin1',
                     dtype={'estadoOrden' : str,
                            'TotalPagado' : np.float64,
                            "Domicilio" : str,
                            "FechaCreaciónOrden" : str,
                            "FechaProgramaciónOrden" : str,
                            "FechadeEntregaOrden" : str,
                            "Ciudad" : str,
                            "fechacreacionProducto" : str,
                            "stado_producto" : str,
                            "Marca" : str,
                            "Departamento" : str,
                            "Pasillo" : str,
                            "Categoria" : str,
                            "ProductoEnPromocion" : np.float64,
                            "cantidadVendida" : np.float64,
                            "PrecioFinal": np.float64})

In [11]:
df_p = df_p.dropna()

In [12]:
%time agg = df_p.groupby(vars_group_b,as_index=False).sum()

CPU times: user 3.56 s, sys: 793 ms, total: 4.35 s
Wall time: 3.77 s


In [13]:
agg.shape

(638906, 12)

In [19]:
agg.to_csv("../Data/AGG_MERQUEO.csv",encoding='latin1',index=False)

## Agregado sub set

In [20]:
%time df = pd.read_csv("../Data/AGG_MERQUEO.csv", encoding='latin1')

CPU times: user 1.09 s, sys: 88 ms, total: 1.18 s
Wall time: 1.08 s


In [21]:
df.columns

Index(['FechaCreaciónOrden', 'Ciudad', 'Departamento', 'estadoOrden',
       'stado_producto', 'Marca', 'Pasillo', 'Categoria',
       'ProductoEnPromocion', 'TotalPagado', 'cantidadVendida', 'PrecioFinal'],
      dtype='object')

In [23]:
df['FechaCreaciónOrden'] = pd.to_datetime(df.FechaCreaciónOrden)

In [24]:
df.set_index("FechaCreaciónOrden",inplace=True)

In [25]:
df_sb = df[df.Ciudad.str.contains("Bogotá")][['cantidadVendida']]
df_sb = df_sb.resample('D').sum()

In [26]:
df.Ciudad.value_counts()

Bogotá      492153
Medellín    146753
Name: Ciudad, dtype: int64